In [1]:
import nltk
import threading
import queue
import feedparser
import uuid



In [2]:
threads = []
queues = [queue.Queue(), queue.Queue()]

In [3]:
def extractWords():
    url = 'https://timesofindia.indiatimes.com/rssfeeds/1081479906.cms'
    feed = feedparser.parse(url)
    for entry in feed['entries'][:5]:
        text = entry['title']
        if 'ex' in text:
            continue
        words = nltk.word_tokenize(text)
        data = {'uuid': uuid.uuid4(), 'input': words}
        queues[0].put(data, True)
        print(">> {} : {}".format(data['uuid'], text))



In [8]:
def extractPOS():
    while True:
        if queues[0].empty():
            break
        else:
            data = queues[0].get()
            words = data['input']
            postags = nltk.pos_tag(words)
            queues[0].task_done()
            queues[1].put({'uuid': data['uuid'], 'input': postags}, True)



In [9]:
def extractNE():
    while True:
        if queues[1].empty():
            break
        else:
            data = queues[1].get()
            postags = data['input']
            queues[1].task_done()
            chunks = nltk.ne_chunk(postags, binary=False)
            print("  << {} : ".format(data['uuid']), end = '')
            for path in chunks:
                try:
                    label = path.label()
                    print(path, end=', ')
                except:
                    pass
            print()



In [10]:
def runProgram():
    e = threading.Thread(target=extractWords())#thread 생성
    e.start()
    threads.append(e)

    p = threading.Thread(target=extractPOS())
    p.start()
    threads.append(p)

    n = threading.Thread(target=extractNE())
    n.start()
    threads.append(n)

    queues[0].join()
    queues[1].join()

    for t in threads:
        t.join()

if __name__ == '__main__':
    runProgram()


>> d1ee4169-35a6-4cf5-a5d5-fa3931f10d09 : SRK & Arbaaz's dad share the same birthday
>> d7a784c9-5c3f-456e-8114-3014fe65ad92 : Salman wishes his "Bhai" SRK on his bday
>> e67554aa-5a0f-40f5-86ff-5f99b4e6af2f : Blast from the past: SRK's unseen photos
>> 75fe8830-52c0-4b67-8f4a-cefb23423181 : Newlyweds who will celebrate their 1st Diwali
>> 147fbb71-d98a-45e2-9b90-c6e1a4740b7e : Twitter shares memes on Anushka-Virat
  << d1ee4169-35a6-4cf5-a5d5-fa3931f10d09 : (ORGANIZATION SRK/NNP), (PERSON Arbaaz/NNP), 
  << d7a784c9-5c3f-456e-8114-3014fe65ad92 : (GPE Salman/NNP), (PERSON Bhai/NNP), 
  << e67554aa-5a0f-40f5-86ff-5f99b4e6af2f : (GPE Blast/NN), (ORGANIZATION SRK/NNP), 
  << 75fe8830-52c0-4b67-8f4a-cefb23423181 : 
  << 147fbb71-d98a-45e2-9b90-c6e1a4740b7e : 
